##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 TensorFlow Cloud 训练 Keras 模型

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/keras/training_keras_models_on_cloud"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">View 在 TensorFlow.org 上查看</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/snapshot-keras/site/en/guide/keras/training_keras_models_on_cloud.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">在 Google Colab 中运行 </a></td>
  <td><a target="_blank" href="https://github.com/keras-team/keras-io/blob/master/guides/training_keras_models_on_cloud.py"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">在 GitHub 中查看源代码</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/keras/training_keras_models_on_cloud.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">下载笔记本</a></td>
</table>

## 简介

[TensorFlow Cloud](https://github.com/tensorflow/cloud) 是一个提供 API 的 Python 软件包，可实现从本地调试到 Google Cloud 中的分布式训练的无缝转换。它简化了将云上的 TensorFlow 模型训练为单个简单函数调用的过程，只需要最少的设置，无需对模型进行任何更改。TensorFlow Cloud 可自动处理特定于云的任务，例如为模型创建 VM 实例和分布策略。本指南将演示如何通过 TensorFlow Cloud 与 Google Cloud 交互，以及 TensorFlow Cloud 中提供的丰富功能。我们从最简单的用例开始。

## 设置

我们从安装 TensorFlow Cloud 并导入本指南中需要的软件包开始。

In [ ]:
!pip install -q tensorflow_cloud

In [ ]:
import tensorflow as tf
import tensorflow_cloud as tfc

from tensorflow import keras
from tensorflow.keras import layers

## API 概述：第一个端到端示例

我们从 Keras 模型训练脚本开始，例如下面的 CNN：

```python
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

model = keras.Sequential(
    [
        keras.Input(shape=(28, 28)),
        # Use a Rescaling layer to make sure input values are in the [0, 1] range.
        layers.experimental.preprocessing.Rescaling(1.0 / 255),
        # The original images have shape (28, 28), so we reshape them to (28, 28, 1)
        layers.Reshape(target_shape=(28, 28, 1)),
        # Follow-up with a classic small convnet
        layers.Conv2D(32, 3, activation="relu"),
        layers.MaxPooling2D(2),
        layers.Conv2D(32, 3, activation="relu"),
        layers.MaxPooling2D(2),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(10),
    ]
)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=keras.metrics.SparseCategoricalAccuracy(),
)

model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.1)
```

要在 Google Cloud 上训练此模型，我们只需在脚本的开头添加对 `run()` 的调用（在导入前）：

```python
tfc.run()
```

使用 TensorFlow Cloud 时，您无需担心特定于云的任务，例如创建 VM 实例和分布策略。此 API 包含所有参数的智能默认值 -- 一切均可配置，但许多模型都可以依赖这些默认值。

调用 `run()` 后，TensorFlow Cloud 将执行以下操作：

- 使您的 Python 脚本或笔记本为分布做好准备。
- 将其转换为具有所需依赖项的 Docker 映像。
- 在 GCP GPU 驱动的 VM 上运行训练作业。
- 以串流方式传输相关日志和作业信息。

默认 VM 配置为 1 个主进程和 0 个工作进程，使用 8 个 CPU 内核和 1 个 Tesla T4 GPU。

## Google Cloud 配置

为了确保为 Cloud 训练提供正确的途径，您将需要完成一些首次设置。如果您是 Google Cloud 的新用户，则需要采取一些预备步骤：

1. 创建一个 GCP 项目；
2. 启用 AI Platform Services；
3. 创建一个服务帐户；
4. 下载授权密钥；
5. 创建一个云存储桶。

可以在 [TensorFlow Cloud 自述文件](https://github.com/tensorflow/cloud#setup-instructions)中找到详细的首次设置说明，并在 [TensorFlow 博客](https://blog.tensorflow.org/2020/08/train-your-tensorflow-model-on-google.html)上查看其他设置示例。

## 常用工作流和云存储

在大多数情况下，您希望在 Google Cloud 上训练后检索模型。为此，在远程训练的同时将保存和加载重定向到 Cloud Storage 至关重要。我们可以将 TensorFlow Cloud 引向我们的 Cloud Storage 桶以执行各种任务。该存储桶可用于保存和加载大型训练数据集、存储回调日志或模型权重以及保存经过训练的模型文件。首先，我们配置 `fit()` 以将模型保存到 Cloud Storage，然后设置 TensorBoard 监控来跟踪训练进度。

In [ ]:
def create_model():
    model = keras.Sequential(
        [
            keras.Input(shape=(28, 28)),
            layers.experimental.preprocessing.Rescaling(1.0 / 255),
            layers.Reshape(target_shape=(28, 28, 1)),
            layers.Conv2D(32, 3, activation="relu"),
            layers.MaxPooling2D(2),
            layers.Conv2D(32, 3, activation="relu"),
            layers.MaxPooling2D(2),
            layers.Conv2D(32, 3, activation="relu"),
            layers.Flatten(),
            layers.Dense(128, activation="relu"),
            layers.Dense(10),
        ]
    )

    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=keras.metrics.SparseCategoricalAccuracy(),
    )
    return model


我们将训练期间生成的 TensorBoard 日志和模型检查点保存到我们的云存储桶中。

In [ ]:
import datetime
import os

# Note: Please change the gcp_bucket to your bucket name.
gcp_bucket = "keras-examples"

checkpoint_path = os.path.join("gs://", gcp_bucket, "mnist_example", "save_at_{epoch}")

tensorboard_path = os.path.join(  # Timestamp included to enable timeseries graphs
    "gs://", gcp_bucket, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
)

callbacks = [
    # TensorBoard will store logs for each epoch and graph performance for us.
    keras.callbacks.TensorBoard(log_dir=tensorboard_path, histogram_freq=1),
    # ModelCheckpoint will save models after each epoch for retrieval later.
    keras.callbacks.ModelCheckpoint(checkpoint_path),
    # EarlyStopping will terminate training when val_loss ceases to improve.
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=3),
]

model = create_model()

这里，我们将直接从 Keras 加载数据。一般而言，最佳做法是将数据集存储到 Cloud Storage 桶中，但是，TensorFlow Cloud 也可以容纳本地存储的数据集。本指南的“多文件”部分中对此进行了介绍。

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

[TensorFlow Cloud](https://github.com/tensorflow/cloud) API 提供了 `remote()` 函数来确定代码是在本地执行还是在云上执行。这样便可为本地和远程执行分别指定 `fit()` 参数，并提供不会导致本地计算机过载的简单调试方法。

In [ ]:
if tfc.remote():
    epochs = 100
    callbacks = callbacks
    batch_size = 128
else:
    epochs = 5
    batch_size = 64
    callbacks = None

model.fit(x_train, y_train, epochs=epochs, callbacks=callbacks, batch_size=batch_size)

训练完成后，我们将模型保存到 GCS 中。

In [ ]:
save_path = os.path.join("gs://", gcp_bucket, "mnist_example")

if tfc.remote():
    model.save(save_path)

此外，我们还可以使用此存储桶来构建 Docker 映像，而不是本地 Docker 实例。为此，只需将您的桶添加到 `docker_image_bucket_name` 参数。

In [ ]:
# docs_infra: no_execute
tfc.run(docker_image_bucket_name=gcp_bucket)

训练模型后，我们可以加载保存的模型并查看 TensorBoard 日志以监视性能。

In [ ]:
# docs_infra: no_execute
model = keras.models.load_model(save_path)

In [ ]:
!#docs_infra: no_execute
!tensorboard dev upload --logdir "gs://keras-examples-jonah/logs/fit" --name "Guide MNIST"

## 大型项目

在许多情况下，包含 Keras 模型的项目可能包含多个 Python 脚本，或者涉及外部数据或特定的依赖项。对于大规模部署，TensorFlow Cloud 拥有完全的灵活性，并提供了许多智能功能来帮助您实现项目。

### 入口点：支持 Python 脚本和 Jupyter 笔记本

您对 `run()` API 的调用不会始终包含在与模型训练代码相同的 Python 脚本中。为此，我们提供了一个 `entry_point` 参数。`entry_point` 参数可用于指定模型训练代码所在的 Python 脚本或笔记本。从与模型相同的脚本中调用 `run()` 时，可使用 `entry_point` 的默认值 `None`。

### `pip` 依赖项

如果您的项目调用其他 `pip` 依赖项，则可以通过包含 `requirements.txt` 文件来指定其他所需的库。在此文件中，只需列出所有必需依赖项的列表，TensorFlow Cloud 随即会将这些依赖项集成到您的云构建中。

### Python 笔记本

TensorFlow Cloud 也可以从 Python 笔记本运行。此外，如果需要，您指定的 `entry_point` 可以是一个笔记本。与脚本相比，需要牢记笔记本上的 TensorFlow Cloud 存在两个关键区别：

- 在笔记本中调用 `run()` 时，必须指定 Cloud Storage 桶来构建和存储 Docker 映像。
- GCloud 身份验证完全通过您的身份验证密钥进行，无需项目规范。本指南的“总结”部分提供了使用笔记本中的 TensorFlow Cloud 的示例工作流。

### 多文件项目

如果您的模型依赖于其他文件，则只需确保这些文件位于指定入口点的同一目录（或子目录）中。与指定 `entry_point` 存储在同一目录中的每个文件，以及与 `entry_point` 相邻的子目录中存储的任何文件，都将包含在 Docker 映像中。对于您可能需要，但无法通过 `pip` 获得的依赖项，情况也是如此

有关具有其他 pip 依赖项的自定义入口点和多文件项目的示例，请在 [TensorFlow Cloud 仓库](https://github.com/tensorflow/cloud/tree/master/src/python/tensorflow_cloud/core/tests/examples/multi_file_example)上查看此多文件示例。为简便起见，我们仅包含示例的 `run()` 调用：

```python
tfc.run(
    docker_image_bucket_name=gcp_bucket,
    entry_point="train_model.py",
    requirements="requirements.txt"
)
```

## 机器配置和分布式训练

模型训练可能需要各种不同的资源，具体取决于模型或数据集的大小。当考虑到具有多个 GPU 的配置时，选择适当的[分配策略](https://www.tensorflow.org/guide/distributed_training)就变得至关重要。在这里，我们概述了一些可能的配置：

### 多工作进程分配

在这里，我们可以使用 `COMMON_MACHINE_CONFIGS` 来指定 1 个首席 CPU 和 4 个工作进程 GPU。

```python
tfc.run(
    docker_image_bucket_name=gcp_bucket,
    chief_config=tfc.COMMON_MACHINE_CONFIGS['CPU'],
    worker_count=2,
    worker_config=tfc.COMMON_MACHINE_CONFIGS['T4_4X']
)
```

默认情况下，TensorFlow Cloud 使用提供的 `chief_config`、`worker_config` 和 `worker_count` 参数通过简单的公式为您的计算机配置选择最佳的分配策略。

- 如果指定的 GPU 数大于零，则将选择 `tf.distribute.MirroredStrategy`。
- 如果工作进程数大于零，则将基于加速器类型选择 `tf.distribute.experimental.MultiWorkerMirroredStrategy` 或 `tf.distribute.experimental.TPUStrategy`。
- 否则，将选择 `tf.distribute.OneDeviceStrategy`。

### TPU 分配

我们在 TPU 上训练相同的模型，如下所示：

```python
tfc.run(
    docker_image_bucket_name=gcp_bucket,
    chief_config=tfc.COMMON_MACHINE_CONFIGS["CPU"],
    worker_count=1,
    worker_config=tfc.COMMON_MACHINE_CONFIGS["TPU"]
)
```

### 自定义分配策略

要指定自定义分配策略，请按照[分布式训练指南](https://www.tensorflow.org/guide/distributed_training)的要求正常格式化代码，然后将 `distribution_strategy` 设置为 `None`。下面，我们将为相同的 MNIST 模型指定我们自己的分配策略。

```python
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
  model = create_model()

if tfc.remote():
    epochs = 100
    batch_size = 128
else:
    epochs = 10
    batch_size = 64
    callbacks = None

model.fit(
    x_train, y_train, epochs=epochs, callbacks=callbacks, batch_size=batch_size
)

tfc.run(
    docker_image_bucket_name=gcp_bucket,
    chief_config=tfc.COMMON_MACHINE_CONFIGS['CPU'],
    worker_count=2,
    worker_config=tfc.COMMON_MACHINE_CONFIGS['T4_4X'],
    distribution_strategy=None
)
```

## 自定义 Docker 映像

默认情况下，TensorFlow Cloud 使用由 Google 提供，与您当前 TensorFlow 版本相对应的 [Docker 基本映像](https://hub.docker.com/r/tensorflow/tensorflow/)。但是，如有必要，您也可以指定一个自定义 Docker 映像来满足您的构建需求。在本示例中，我们将从旧版本的 TensorFlow 中指定 Docker 映像：

```python
tfc.run(
    docker_image_bucket_name=gcp_bucket,
    base_docker_image="tensorflow/tensorflow:2.1.0-gpu"
)
```

## 其他指标

您可能会发现，使用特定标签标记 Cloud 作业或在 Cloud 训练期间流式传输模型的日志非常有用。良好的做法是在所有 Cloud 作业上维护适当的标签，以便做好记录。为此，`run()` 接受最多包含 64 个键值对的标签字典，可从 Cloud 构建日志中查看这些键值对。可以使用执行 `tfc.run` 所提供的链接来访问诸如周期性能和模型保存内部构件等日志，也可以使用 `stream_logs` 标记将此类日志打印到本地终端。

```python
job_labels = {"job": "mnist-example", "team": "keras-io", "user": "jonah"}

tfc.run(
    docker_image_bucket_name=gcp_bucket,
    job_labels=job_labels,
    stream_logs=True
)
```

## 总结

对于使用本指南中介绍的许多功能的深入 Colab，请按照[本示例](https://github.com/tensorflow/cloud/blob/master/src/python/tensorflow_cloud/core/tests/examples/dogs_classification.ipynb)进行操作来训练一个最新模型，以便利用特征提取识别照片中狗的品种。